This file forecasts derailment/subreddit using BERT.

# 1. Install requirements

In [80]:
model_name = "bert-base-uncased"
test_size = 0.2
labels = [True, False]  # replace with your real labels
num_labels = len(labels)
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# 2. Read and Prepare the Dataset

In [81]:
import pandas as pd
import pickle
import os

In [82]:
corpus = os.path.join('..', '..', 'Data', 'processed', 'email_label_dict_balanced.pkl')

In [83]:
# Load the balanced dictionary
with open(corpus, 'rb') as f:
    emails_and_tag = pickle.load(f)

print(f"Loaded {len(emails_and_tag):,} emails")

# Check class distribution
casual_count = sum(1 for v in emails_and_tag.values() if v == 0)
business_count = sum(1 for v in emails_and_tag.values() if v == 1)
print(f"Casual: {casual_count:,} | Business: {business_count:,}")
print(f"Ratio: 1:{business_count/casual_count:.1f}")


Loaded 32,418 emails
Casual: 5,403 | Business: 27,015
Ratio: 1:5.0


In [84]:
import random

emails = []
is_business = []

for key, value in emails_and_tag.items():
    emails.append(key)
    is_business.append(value) 


# Zip the lists together
combined = list(zip(emails, is_business))

# Shuffle the combined list
random.shuffle(combined)

# Unzip back into two lists
email_shuffled, is_buisness_shuffled = zip(*combined)

# Convert back to lists (optional)
email_shuffled = list(email_shuffled)
is_buisness_shuffled = list(is_buisness_shuffled)

pre_dataframe = {"email": email_shuffled, "labels":is_buisness_shuffled}
df = pd.DataFrame(pre_dataframe)

In [85]:
df.head()

,email,labels
0,Subject: Fundies meeting\n\nAre we going to co...,1
1,Subject: FoolWatch: Can You Save Priceline?\n\n,1
2,Subject: Certificate application update (710...,1
3,Subject: History of the tilde\n\nURL: [URL]\nD...,0
4,"Subject: RE: when you are ready\n\nCarole,\nWe...",1


### Label Encoder

In [86]:
from sklearn import preprocessing

In [87]:
# le = preprocessing.LabelEncoder()
# le.fit(df["derails"].tolist())
# df['label'] = le.transform(df["derails"].tolist())

In [88]:
df.head()

,email,labels
0,Subject: Fundies meeting\n\nAre we going to co...,1
1,Subject: FoolWatch: Can You Save Priceline?\n\n,1
2,Subject: Certificate application update (710...,1
3,Subject: History of the tilde\n\nURL: [URL]\nD...,0
4,"Subject: RE: when you are ready\n\nCarole,\nWe...",1


### Train/Test/Val Split

In [89]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [90]:
# First split off 20% for test
df_train_val, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Now split the remaining 80% into 60% train and 20% val
# 20% out of the remaining 80% is 0.25 of that chunk
df_train, df_val = train_test_split(df_train_val, test_size=0.25, random_state=42)

# Hugging Face format
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop=True)),
    "validation": Dataset.from_pandas(df_val.reset_index(drop=True)),
    "test": Dataset.from_pandas(df_test.reset_index(drop=True)),
})


### Convert to correct dataframe

In [91]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]


### Tokenizer

In [92]:
from transformers import AutoTokenizer

In [93]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["email"], truncation=True)

In [94]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 19450/19450 [00:03<00:00, 5208.87 examples/s]


In [95]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 6484/6484 [00:00<00:00, 6806.63 examples/s]


# 3. Initialize Model

In [96]:
from transformers import AutoModelForSequenceClassification

In [97]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 4. Train model

In [98]:
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

In [99]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [100]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = metric.compute(predictions=predictions, references=labels)["accuracy"]
    return {"eval_accuracy": acc}

In [102]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    logging_strategy="epoch",
    eval_strategy="epoch",                 # ✅ clean & modern
    save_strategy="epoch",                 # ✅ matches eval
    save_total_limit=2,                    # ✅ keeps best 2 checkpoints
    load_best_model_at_end=True,           # ✅ will restore best version
    metric_for_best_model="eval_accuracy", # ✅ or use "eval_accuracy"
    greater_is_better=True,                # ✅ for accuracy
    report_to="none",
    learning_rate=3e-5,
    per_device_train_batch_size=128,       # for speed
    per_device_eval_batch_size=256,        # speed
    fp16=True,                             # speed
    warmup_steps=500,
    weight_decay=0.05,
)

trainer = Trainer(
    # model=model,
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]  # 👈 Add this line
)


/tmp/ipykernel_2188714/2788371201.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [103]:
def compute_objective(metrics):
    return metrics["eval_accuracy"]


Now re-generate the model with the best hyperparameters

In [104]:
# 4. Retrain and save
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.381800,0.205436,0.938618
2,0.177600,0.156254,0.947872
3,0.139500,0.134937,0.953115
4,0.096300,0.153828,0.954812
5,0.057300,0.170560,0.957588
6,0.030100,0.198582,0.946175
7,0.016900,0.204742,0.956663
8,0.010200,0.225544,0.958205
9,0.006000,0.231204,0.958359
10,0.003700,0.233154,0.958667


TrainOutput(global_step=1520, training_loss=0.09194409733540133, metrics={'train_runtime': 458.4031, 'train_samples_per_second': 424.299, 'train_steps_per_second': 3.316, 'total_flos': 5.117510026752e+16, 'train_loss': 0.09194409733540133, 'epoch': 10.0})

In [105]:
# trainer.save_model('BERT')

# 5. Evaluate Model

In [106]:
from sklearn.metrics import classification_report

In [108]:
preds = trainer.predict(tokenized_train)
preds = np.argmax(preds[:3][0],axis=1)
GT = df_train['labels'].tolist()
print(classification_report(GT,preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3301
           1       1.00      1.00      1.00     16149

    accuracy                           1.00     19450
   macro avg       1.00      1.00      1.00     19450
weighted avg       1.00      1.00      1.00     19450



In [109]:
output = trainer.predict(tokenized_test)
logits = output.predictions  # raw model output before argmax
preds = np.argmax(logits, axis=1)  # class with highest score

GT = df_val['labels'].tolist()  
print(classification_report(GT, preds))


              precision    recall  f1-score   support

           0       0.17      0.14      0.15      1073
           1       0.83      0.86      0.85      5411

    accuracy                           0.74      6484
   macro avg       0.50      0.50      0.50      6484
weighted avg       0.72      0.74      0.73      6484



In [110]:
trainer.evaluate()

{'eval_accuracy': 0.958667489204195,
 'eval_loss': 0.23315392434597015,
 'eval_runtime': 5.5397,
 'eval_samples_per_second': 1170.47,
 'eval_steps_per_second': 4.693,
 'epoch': 10.0}

In [ ]:
trainer.save_model("./my_email_classifier")
tokenizer.save_pretrained("./my_email_classifier")